In [1]:
!pip install selenium beautifulsoup4 requests

  Using cached selenium-4.38.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.32.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached websocket_client-1.9.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.4-cp312-cp312-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-2.0.0-cp312-cp31


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install webdriver-manager


  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#!/usr/bin/env python3
"""
Google Maps to Business Website Contact Info Scraper
Extracts emails, phones, and social media from business websites found on Google Maps.
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


class BusinessInfoScraper:
    """Scrapes business contact information from Google Maps and business websites."""
    
    # Regex patterns
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    
    # Social media domains
    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }
    
    def __init__(self, headless: bool = True):
        """Initialize the scraper with Selenium options."""
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
    
    def _init_driver(self) -> webdriver.Chrome:
        """Initialize Chrome WebDriver with appropriate options."""
        options = webdriver.ChromeOptions()
        if self.headless:
            options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        
        return webdriver.Chrome(options=options)
    
    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        """
        Extract the business website URL from Google Maps listing.
        
        Args:
            gmaps_url: Google Maps business listing URL
            
        Returns:
            Business website URL or None if not found
        """
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            
            # Wait for page to load and find website button
            wait = WebDriverWait(driver, 15)
            
            # Try multiple selectors for the website button
            selectors = [
                "a[data-item-id='authority']",
                "a[aria-label*='Website']",
                "a[href*='http']:has-text('Website')",
                "//a[contains(@aria-label, 'Website')]",
                "//a[contains(., 'Website')]"
            ]
            
            website_url = None
            for selector in selectors:
                try:
                    if selector.startswith('//'):
                        element = wait.until(
                            EC.presence_of_element_located((By.XPATH, selector))
                        )
                    else:
                        element = wait.until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                        )
                    
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # Clean up Google redirect URLs
                        if 'google.com/url?' in website_url:
                            match = re.search(r'[?&]q=([^&]+)', website_url)
                            if match:
                                from urllib.parse import unquote
                                website_url = unquote(match.group(1))
                        break
                except (TimeoutException, NoSuchElementException):
                    continue
            
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found on Google Maps listing")
                return None
                
        except Exception as e:
            print(f"❌ Error extracting website from Google Maps: {e}")
            return None
        finally:
            if driver:
                driver.quit()
    
    def _extract_emails(self, text: str) -> Set[str]:
        """Extract email addresses from text."""
        emails = self.EMAIL_PATTERN.findall(text)
        # Filter out common false positives
        filtered = {
            email.lower() for email in emails
            if not email.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.svg'))
        }
        return filtered
    
    def _extract_phones(self, text: str) -> Set[str]:
        """Extract phone numbers from text."""
        phones = self.PHONE_PATTERN.findall(text)
        # Clean and filter phone numbers
        cleaned = set()
        for phone in phones:
            # Remove excessive spaces/dashes
            clean = re.sub(r'\s+', ' ', phone.strip())
            # Only keep if it has enough digits
            digit_count = sum(c.isdigit() for c in clean)
            if digit_count >= 9:  # Minimum valid phone number length
                cleaned.add(clean)
        return cleaned
    
    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        """Extract social media links from page."""
        social_links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        
        for link in soup.find_all('a', href=True):
            href = link['href'].lower()
            
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(domain in href for domain in domains):
                    full_url = urljoin(base_url, link['href'])
                    if full_url not in social_links[platform]:
                        social_links[platform].append(full_url)
        
        return social_links
    
    def _find_contact_pages(self, soup: BeautifulSoup, base_url: str) -> List[str]:
        """Find links to contact and about pages."""
        contact_pages = []
        keywords = ['contact', 'about', 'reach', 'connect', 'support']
        
        for link in soup.find_all('a', href=True):
            href = link['href'].lower()
            text = link.get_text().lower()
            
            # Check if link contains contact-related keywords
            if any(keyword in href or keyword in text for keyword in keywords):
                full_url = urljoin(base_url, link['href'])
                parsed = urlparse(full_url)
                
                # Only include internal links
                if parsed.netloc == urlparse(base_url).netloc:
                    if full_url not in contact_pages and full_url != base_url:
                        contact_pages.append(full_url)
        
        return contact_pages[:5]  # Limit to 5 additional pages
    
    def _scrape_page(self, url: str) -> tuple:
        """
        Scrape a single page for contact information.
        
        Returns:
            Tuple of (emails, phones, soup object)
        """
        try:
            print(f"📄 Scraping: {url}")
            response = self.session.get(url, timeout=10, allow_redirects=True)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Remove script and style elements
            for element in soup(['script', 'style', 'noscript']):
                element.decompose()
            
            text = soup.get_text()
            
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            
            return emails, phones, soup
            
        except Exception as e:
            print(f"⚠️  Error scraping {url}: {e}")
            return set(), set(), None
    
    def scrape_business_website(self, website_url: str) -> Dict:
        """
        Scrape business website for contact information.
        
        Args:
            website_url: Business website URL
            
        Returns:
            Dictionary containing all extracted information
        """
        print(f"\n🌐 Starting website scrape: {website_url}")
        
        # Initialize result structure
        result = {
            'website': website_url,
            'emails': [],
            'phones': [],
            'social_links': {platform: [] for platform in self.SOCIAL_PLATFORMS},
            'additional_pages': []
        }
        
        all_emails = set()
        all_phones = set()
        pages_to_scrape = [website_url]
        scraped_pages = set()
        
        # Scrape main page
        emails, phones, soup = self._scrape_page(website_url)
        all_emails.update(emails)
        all_phones.update(phones)
        scraped_pages.add(website_url)
        
        if soup:
            # Extract social media links from main page
            social_links = self._extract_social_links(soup, website_url)
            for platform, links in social_links.items():
                result['social_links'][platform].extend(links)
            
            # Find contact pages
            contact_pages = self._find_contact_pages(soup, website_url)
            result['additional_pages'] = contact_pages
            
            # Scrape additional pages
            for page_url in contact_pages:
                if page_url not in scraped_pages:
                    time.sleep(1)  # Polite delay
                    emails, phones, page_soup = self._scrape_page(page_url)
                    all_emails.update(emails)
                    all_phones.update(phones)
                    scraped_pages.add(page_url)
                    
                    # Extract social links from additional pages
                    if page_soup:
                        social_links = self._extract_social_links(page_soup, website_url)
                        for platform, links in social_links.items():
                            for link in links:
                                if link not in result['social_links'][platform]:
                                    result['social_links'][platform].append(link)
        
        # Deduplicate and sort results
        result['emails'] = sorted(list(all_emails))
        result['phones'] = sorted(list(all_phones))
        
        # Remove duplicate social links
        for platform in result['social_links']:
            result['social_links'][platform] = list(set(result['social_links'][platform]))
        
        print(f"\n✅ Scraping complete!")
        print(f"   Emails found: {len(result['emails'])}")
        print(f"   Phones found: {len(result['phones'])}")
        print(f"   Social links: {sum(len(v) for v in result['social_links'].values())}")
        
        return result
    
    def scrape_from_google_maps(self, gmaps_url: str) -> Optional[Dict]:
        """
        Complete pipeline: Google Maps → Website → Contact Info.
        
        Args:
            gmaps_url: Google Maps business listing URL
            
        Returns:
            Dictionary with all extracted information or None if website not found
        """
        # Step 1: Get website from Google Maps
        website_url = self.get_website_from_google_maps(gmaps_url)
        
        if not website_url:
            return None
        
        # Step 2: Scrape the business website
        return self.scrape_business_website(website_url)
    
    def save_to_json(self, data: Dict, filename: str = 'business_info.json'):
        """Save scraped data to JSON file."""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            print(f"\n💾 Data saved to {filename}")
        except Exception as e:
            print(f"❌ Error saving to JSON: {e}")


def main():
    """Main execution function."""
    print("=" * 70)
    print("🗺️  GOOGLE MAPS BUSINESS INFO SCRAPER")
    print("=" * 70)
    
    # Get Google Maps URL from user
    gmaps_url = input("\n📍 Enter Google Maps business URL: ").strip()
    
    if not gmaps_url:
        print("❌ No URL provided. Exiting.")
        return
    
    # Initialize scraper
    scraper = BusinessInfoScraper(headless=True)
    
    try:
        # Run the complete scraping pipeline
        result = scraper.scrape_from_google_maps(gmaps_url)
        
        if result:
            # Display results
            print("\n" + "=" * 70)
            print("📊 RESULTS")
            print("=" * 70)
            print(json.dumps(result, indent=2, ensure_ascii=False))
            
            # Save to file
            scraper.save_to_json(result)
        else:
            print("\n❌ Could not find business website on Google Maps listing.")
            
    except Exception as e:
        print(f"\n❌ An error occurred: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

🗺️  GOOGLE MAPS BUSINESS INFO SCRAPER
🔍 Opening Google Maps: https://www.google.com/maps/place/Broadway+Pizza+-+Satellite+Town/@33.6333227,73.0342839,14z/data=!4m6!3m5!1s0x38df95373f97dfa5:0xf9c07921bce37c26!8m2!3d33.6333227!4d73.0703328!16s%2Fg%2F11ppw3hd9k?authuser=0&entry=ttu&g_ep=EgoyMDI1MTEwNC4xIKXMDSoASAFQAw%3D%3D
✅ Found website: http://www.broadwaypizza.com.pk/

🌐 Starting website scrape: http://www.broadwaypizza.com.pk/
📄 Scraping: http://www.broadwaypizza.com.pk/
📄 Scraping: http://www.broadwaypizza.com.pk/about/
⚠️  Error scraping http://www.broadwaypizza.com.pk/about/: 404 Client Error: Not Found for url: http://www.broadwaypizza.com.pk/about/
📄 Scraping: http://www.broadwaypizza.com.pk/contact
⚠️  Error scraping http://www.broadwaypizza.com.pk/contact: 404 Client Error: Not Found for url: http://www.broadwaypizza.com.pk/contact

✅ Scraping complete!
   Emails found: 0
   Phones found: 0
   Social links: 0

📊 RESULTS
{
  "website": "http://www.broadwaypizza.com.pk/",
  "ema

In [3]:
#!/usr/bin/env python3
"""
Google Maps to Business Website Contact Info Scraper
Extracts emails, phones, and social media from business websites found on Google Maps.
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


class BusinessInfoScraper:
    """Scrapes business contact information from Google Maps and business websites."""
    
    # Regex patterns
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    
    # Social media domains
    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }
    
    def __init__(self, headless: bool = True):
        """Initialize the scraper with Selenium options."""
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
    
    def _init_driver(self) -> webdriver.Chrome:
        """Initialize Chrome WebDriver with appropriate options."""
        options = webdriver.ChromeOptions()
        if self.headless:
            options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        
        return webdriver.Chrome(options=options)
    
    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        """
        Extract the business website URL from Google Maps listing.
        
        Args:
            gmaps_url: Google Maps business listing URL
            
        Returns:
            Business website URL or None if not found
        """
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            
            # Wait for page to load and find website button
            wait = WebDriverWait(driver, 15)
            
            # Try multiple selectors for the website button
            selectors = [
                "a[data-item-id='authority']",
                "a[aria-label*='Website']",
                "a[href*='http']:has-text('Website')",
                "//a[contains(@aria-label, 'Website')]",
                "//a[contains(., 'Website')]"
            ]
            
            website_url = None
            for selector in selectors:
                try:
                    if selector.startswith('//'):
                        element = wait.until(
                            EC.presence_of_element_located((By.XPATH, selector))
                        )
                    else:
                        element = wait.until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                        )
                    
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # Clean up Google redirect URLs
                        if 'google.com/url?' in website_url:
                            match = re.search(r'[?&]q=([^&]+)', website_url)
                            if match:
                                from urllib.parse import unquote
                                website_url = unquote(match.group(1))
                        break
                except (TimeoutException, NoSuchElementException):
                    continue
            
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found on Google Maps listing")
                return None
                
        except Exception as e:
            print(f"❌ Error extracting website from Google Maps: {e}")
            return None
        finally:
            if driver:
                driver.quit()
    
    def _extract_emails(self, text: str) -> Set[str]:
        """Extract email addresses from text."""
        emails = self.EMAIL_PATTERN.findall(text)
        # Filter out common false positives
        filtered = {
            email.lower() for email in emails
            if not email.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.svg'))
        }
        return filtered
    
    def _extract_phones(self, text: str) -> Set[str]:
        """Extract phone numbers from text."""
        phones = self.PHONE_PATTERN.findall(text)
        # Clean and filter phone numbers
        cleaned = set()
        for phone in phones:
            # Remove excessive spaces/dashes
            clean = re.sub(r'\s+', ' ', phone.strip())
            # Only keep if it has enough digits
            digit_count = sum(c.isdigit() for c in clean)
            if digit_count >= 9:  # Minimum valid phone number length
                cleaned.add(clean)
        return cleaned
    
    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        """Extract social media links from page."""
        social_links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        
        for link in soup.find_all('a', href=True):
            href = link['href'].lower()
            
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(domain in href for domain in domains):
                    full_url = urljoin(base_url, link['href'])
                    if full_url not in social_links[platform]:
                        social_links[platform].append(full_url)
        
        return social_links
    
    def _find_contact_pages(self, soup: BeautifulSoup, base_url: str) -> List[str]:
        """Find links to contact and about pages."""
        contact_pages = []
        keywords = ['contact', 'about', 'reach', 'connect', 'support']
        
        for link in soup.find_all('a', href=True):
            href = link['href'].lower()
            text = link.get_text().lower()
            
            # Check if link contains contact-related keywords
            if any(keyword in href or keyword in text for keyword in keywords):
                full_url = urljoin(base_url, link['href'])
                parsed = urlparse(full_url)
                
                # Only include internal links
                if parsed.netloc == urlparse(base_url).netloc:
                    if full_url not in contact_pages and full_url != base_url:
                        contact_pages.append(full_url)
        
        return contact_pages[:5]  # Limit to 5 additional pages
    
    def _scrape_page(self, url: str) -> tuple:
        """
        Scrape a single page for contact information.
        
        Returns:
            Tuple of (emails, phones, soup object)
        """
        try:
            print(f"📄 Scraping: {url}")
            response = self.session.get(url, timeout=10, allow_redirects=True)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Remove script and style elements
            for element in soup(['script', 'style', 'noscript']):
                element.decompose()
            
            text = soup.get_text()
            
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            
            return emails, phones, soup
            
        except Exception as e:
            print(f"⚠️  Error scraping {url}: {e}")
            return set(), set(), None
    
    def scrape_business_website(self, website_url: str) -> Dict:
        """
        Scrape business website for contact information.
        
        Args:
            website_url: Business website URL
            
        Returns:
            Dictionary containing all extracted information
        """
        print(f"\n🌐 Starting website scrape: {website_url}")
        
        # Initialize result structure
        result = {
            'website': website_url,
            'emails': [],
            'phones': [],
            'social_links': {platform: [] for platform in self.SOCIAL_PLATFORMS},
            'additional_pages': []
        }
        
        all_emails = set()
        all_phones = set()
        pages_to_scrape = [website_url]
        scraped_pages = set()
        
        # Scrape main page
        emails, phones, soup = self._scrape_page(website_url)
        all_emails.update(emails)
        all_phones.update(phones)
        scraped_pages.add(website_url)
        
        if soup:
            # Extract social media links from main page
            social_links = self._extract_social_links(soup, website_url)
            for platform, links in social_links.items():
                result['social_links'][platform].extend(links)
            
            # Find contact pages
            contact_pages = self._find_contact_pages(soup, website_url)
            result['additional_pages'] = contact_pages
            
            # Scrape additional pages
            for page_url in contact_pages:
                if page_url not in scraped_pages:
                    time.sleep(1)  # Polite delay
                    emails, phones, page_soup = self._scrape_page(page_url)
                    all_emails.update(emails)
                    all_phones.update(phones)
                    scraped_pages.add(page_url)
                    
                    # Extract social links from additional pages
                    if page_soup:
                        social_links = self._extract_social_links(page_soup, website_url)
                        for platform, links in social_links.items():
                            for link in links:
                                if link not in result['social_links'][platform]:
                                    result['social_links'][platform].append(link)
        
        # Deduplicate and sort results
        result['emails'] = sorted(list(all_emails))
        result['phones'] = sorted(list(all_phones))
        
        # Remove duplicate social links
        for platform in result['social_links']:
            result['social_links'][platform] = list(set(result['social_links'][platform]))
        
        print(f"\n✅ Scraping complete!")
        print(f"   Emails found: {len(result['emails'])}")
        print(f"   Phones found: {len(result['phones'])}")
        print(f"   Social links: {sum(len(v) for v in result['social_links'].values())}")
        
        return result
    
    def scrape_from_google_maps(self, gmaps_url: str) -> Optional[Dict]:
        """
        Complete pipeline: Google Maps → Website → Contact Info.
        
        Args:
            gmaps_url: Google Maps business listing URL
            
        Returns:
            Dictionary with all extracted information or None if website not found
        """
        # Step 1: Get website from Google Maps
        website_url = self.get_website_from_google_maps(gmaps_url)
        
        if not website_url:
            return None
        
        # Step 2: Scrape the business website
        return self.scrape_business_website(website_url)
    
    def save_to_json(self, data: Dict, filename: str = 'business_info.json'):
        """Save scraped data to JSON file."""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            print(f"\n💾 Data saved to {filename}")
        except Exception as e:
            print(f"❌ Error saving to JSON: {e}")


def main():
    """Main execution function."""
    print("=" * 70)
    print("🗺️  GOOGLE MAPS BUSINESS INFO SCRAPER")
    print("=" * 70)
    
    # Get Google Maps URL from user
    gmaps_url = input("\n📍 Enter Google Maps business URL: ").strip()
    
    if not gmaps_url:
        print("❌ No URL provided. Exiting.")
        return
    
    # Initialize scraper
    scraper = BusinessInfoScraper(headless=True)
    
    try:
        # Run the complete scraping pipeline
        result = scraper.scrape_from_google_maps(gmaps_url)
        
        if result:
            # Display results
            print("\n" + "=" * 70)
            print("📊 RESULTS")
            print("=" * 70)
            print(json.dumps(result, indent=2, ensure_ascii=False))
            
            # Save to file
            scraper.save_to_json(result)
        else:
            print("\n❌ Could not find business website on Google Maps listing.")
            
    except Exception as e:
        print(f"\n❌ An error occurred: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

🗺️  GOOGLE MAPS BUSINESS INFO SCRAPER
🔍 Opening Google Maps: https://www.google.com/maps/place/Capital+Pizza/@33.6333227,73.0342839,14z/data=!4m6!3m5!1s0x38df95043e2bff5d:0x260f799fbba0b76f!8m2!3d33.6229741!4d73.0616476!16s%2Fg%2F11w1klzrk6?authuser=0&entry=ttu&g_ep=EgoyMDI1MTEwNC4xIKXMDSoASAFQAw%3D%3D
✅ Found website: https://capitalpizza.pk/

🌐 Starting website scrape: https://capitalpizza.pk/
📄 Scraping: https://capitalpizza.pk/
📄 Scraping: https://capitalpizza.pk/about-us/
📄 Scraping: https://capitalpizza.pk/contact/

✅ Scraping complete!
   Emails found: 2
   Phones found: 1
   Social links: 4

📊 RESULTS
{
  "website": "https://capitalpizza.pk/",
  "emails": [
    "info@capitalpizza.pk",
    "order@capitalpizza.pk"
  ],
  "phones": [
    "0318 893 000 3"
  ],
  "social_links": {
    "facebook": [
      "https://www.facebook.com/Capitalpizza.pk"
    ],
    "instagram": [
      "https://www.instagram.com/capitalpizza.pk/"
    ],
    "linkedin": [
      "https://www.linkedin.com/comp

In [4]:
#!/usr/bin/env python3
"""
Google Maps to Business Website Contact Info Scraper
Extracts emails, phones, and social media from business websites found on Google Maps.
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


class BusinessInfoScraper:
    """Scrapes business contact information from Google Maps and business websites."""
    
    # Regex patterns
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    
    # Social media domains
    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }
    
    def __init__(self, headless: bool = True):
        """Initialize the scraper with Selenium options."""
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })
    
    def _init_driver(self) -> webdriver.Chrome:
        """Initialize Chrome WebDriver with appropriate options."""
        options = webdriver.ChromeOptions()
        if self.headless:
            options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        
        return webdriver.Chrome(options=options)
    
    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        """
        Extract the business website URL from Google Maps listing.
        
        Args:
            gmaps_url: Google Maps business listing URL
            
        Returns:
            Business website URL or None if not found
        """
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            
            # Wait for page to load and find website button
            wait = WebDriverWait(driver, 15)
            
            # Try multiple selectors for the website button
            selectors = [
                "a[data-item-id='authority']",
                "a[aria-label*='Website']",
                "a[href*='http']:has-text('Website')",
                "//a[contains(@aria-label, 'Website')]",
                "//a[contains(., 'Website')]"
            ]
            
            website_url = None
            for selector in selectors:
                try:
                    if selector.startswith('//'):
                        element = wait.until(
                            EC.presence_of_element_located((By.XPATH, selector))
                        )
                    else:
                        element = wait.until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                        )
                    
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # Clean up Google redirect URLs
                        if 'google.com/url?' in website_url:
                            match = re.search(r'[?&]q=([^&]+)', website_url)
                            if match:
                                from urllib.parse import unquote
                                website_url = unquote(match.group(1))
                        break
                except (TimeoutException, NoSuchElementException):
                    continue
            
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found on Google Maps listing")
                return None
                
        except Exception as e:
            print(f"❌ Error extracting website from Google Maps: {e}")
            return None
        finally:
            if driver:
                driver.quit()
    
    def _extract_emails(self, text: str) -> Set[str]:
        """Extract email addresses from text."""
        emails = self.EMAIL_PATTERN.findall(text)
        # Filter out common false positives
        filtered = {
            email.lower() for email in emails
            if not email.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.svg'))
        }
        return filtered
    
    def _extract_phones(self, text: str) -> Set[str]:
        """Extract phone numbers from text."""
        phones = self.PHONE_PATTERN.findall(text)
        # Clean and filter phone numbers
        cleaned = set()
        for phone in phones:
            # Remove excessive spaces/dashes
            clean = re.sub(r'\s+', ' ', phone.strip())
            # Only keep if it has enough digits
            digit_count = sum(c.isdigit() for c in clean)
            if digit_count >= 9:  # Minimum valid phone number length
                cleaned.add(clean)
        return cleaned
    
    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        """Extract social media links from page."""
        social_links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        
        for link in soup.find_all('a', href=True):
            href = link['href'].lower()
            
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(domain in href for domain in domains):
                    full_url = urljoin(base_url, link['href'])
                    if full_url not in social_links[platform]:
                        social_links[platform].append(full_url)
        
        return social_links
    
    def _find_contact_pages(self, soup: BeautifulSoup, base_url: str) -> List[str]:
        """Find links to contact and about pages."""
        contact_pages = []
        keywords = ['contact', 'about', 'reach', 'connect', 'support']
        
        for link in soup.find_all('a', href=True):
            href = link['href'].lower()
            text = link.get_text().lower()
            
            # Check if link contains contact-related keywords
            if any(keyword in href or keyword in text for keyword in keywords):
                full_url = urljoin(base_url, link['href'])
                parsed = urlparse(full_url)
                
                # Only include internal links
                if parsed.netloc == urlparse(base_url).netloc:
                    if full_url not in contact_pages and full_url != base_url:
                        contact_pages.append(full_url)
        
        return contact_pages[:5]  # Limit to 5 additional pages
    
    def _scrape_page(self, url: str) -> tuple:
        """
        Scrape a single page for contact information.
        
        Returns:
            Tuple of (emails, phones, soup object)
        """
        try:
            print(f"📄 Scraping: {url}")
            response = self.session.get(url, timeout=10, allow_redirects=True)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Remove script and style elements
            for element in soup(['script', 'style', 'noscript']):
                element.decompose()
            
            text = soup.get_text()
            
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            
            return emails, phones, soup
            
        except Exception as e:
            print(f"⚠️  Error scraping {url}: {e}")
            return set(), set(), None
    
    def scrape_business_website(self, website_url: str) -> Dict:
        """
        Scrape business website for contact information.
        
        Args:
            website_url: Business website URL
            
        Returns:
            Dictionary containing all extracted information
        """
        print(f"\n🌐 Starting website scrape: {website_url}")
        
        # Initialize result structure
        result = {
            'website': website_url,
            'emails': [],
            'phones': [],
            'social_links': {platform: [] for platform in self.SOCIAL_PLATFORMS},
            'additional_pages': []
        }
        
        all_emails = set()
        all_phones = set()
        pages_to_scrape = [website_url]
        scraped_pages = set()
        
        # Scrape main page
        emails, phones, soup = self._scrape_page(website_url)
        all_emails.update(emails)
        all_phones.update(phones)
        scraped_pages.add(website_url)
        
        if soup:
            # Extract social media links from main page
            social_links = self._extract_social_links(soup, website_url)
            for platform, links in social_links.items():
                result['social_links'][platform].extend(links)
            
            # Find contact pages
            contact_pages = self._find_contact_pages(soup, website_url)
            result['additional_pages'] = contact_pages
            
            # Scrape additional pages
            for page_url in contact_pages:
                if page_url not in scraped_pages:
                    time.sleep(1)  # Polite delay
                    emails, phones, page_soup = self._scrape_page(page_url)
                    all_emails.update(emails)
                    all_phones.update(phones)
                    scraped_pages.add(page_url)
                    
                    # Extract social links from additional pages
                    if page_soup:
                        social_links = self._extract_social_links(page_soup, website_url)
                        for platform, links in social_links.items():
                            for link in links:
                                if link not in result['social_links'][platform]:
                                    result['social_links'][platform].append(link)
        
        # Deduplicate and sort results
        result['emails'] = sorted(list(all_emails))
        result['phones'] = sorted(list(all_phones))
        
        # Remove duplicate social links
        for platform in result['social_links']:
            result['social_links'][platform] = list(set(result['social_links'][platform]))
        
        print(f"\n✅ Scraping complete!")
        print(f"   Emails found: {len(result['emails'])}")
        print(f"   Phones found: {len(result['phones'])}")
        print(f"   Social links: {sum(len(v) for v in result['social_links'].values())}")
        
        return result
    
    def scrape_from_google_maps(self, gmaps_url: str) -> Optional[Dict]:
        """
        Complete pipeline: Google Maps → Website → Contact Info.
        
        Args:
            gmaps_url: Google Maps business listing URL
            
        Returns:
            Dictionary with all extracted information or None if website not found
        """
        # Step 1: Get website from Google Maps
        website_url = self.get_website_from_google_maps(gmaps_url)
        
        if not website_url:
            return None
        
        # Step 2: Scrape the business website
        return self.scrape_business_website(website_url)
    
    def save_to_json(self, data: Dict, filename: str = 'business_info.json'):
        """Save scraped data to JSON file."""
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            print(f"\n💾 Data saved to {filename}")
        except Exception as e:
            print(f"❌ Error saving to JSON: {e}")


def main():
    """Main execution function."""
    print("=" * 70)
    print("🗺️  GOOGLE MAPS BUSINESS INFO SCRAPER")
    print("=" * 70)
    
    # Get Google Maps URL from user
    gmaps_url = input("\n📍 Enter Google Maps business URL: ").strip()
    
    if not gmaps_url:
        print("❌ No URL provided. Exiting.")
        return
    
    # Initialize scraper
    scraper = BusinessInfoScraper(headless=True)
    
    try:
        # Run the complete scraping pipeline
        result = scraper.scrape_from_google_maps(gmaps_url)
        
        if result:
            # Display results
            print("\n" + "=" * 70)
            print("📊 RESULTS")
            print("=" * 70)
            print(json.dumps(result, indent=2, ensure_ascii=False))
            
            # Save to file
            scraper.save_to_json(result)
        else:
            print("\n❌ Could not find business website on Google Maps listing.")
            
    except Exception as e:
        print(f"\n❌ An error occurred: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

🗺️  GOOGLE MAPS BUSINESS INFO SCRAPER
🔍 Opening Google Maps: https://www.google.com/maps/place/Saffron+Foodies/@33.6333907,73.0342839,14z/data=!4m7!3m6!1s0x38df95209a7d563f:0x97f7b42c71db5bc3!8m2!3d33.6381307!4d73.0682772!15sCgpyZXN0YXVyYW50WgwiCnJlc3RhdXJhbnSSAQpyZXN0YXVyYW50mgEjQ2haRFNVaE5NRzluUzBWSlEwRm5TVVIxTms5MU5HWjNFQUXgAQD6AQUIqgIQSg!16s%2Fg%2F11c30vmh9h?authuser=0&coh=277533&entry=tts&g_ep=EgoyMDI1MTEwNC4xIPu8ASoASAFQAw%3D%3D&skid=e79c76e9-1573-4518-bd64-36cec7f049a0
✅ Found website: http://www.saffron.com.pk/

🌐 Starting website scrape: http://www.saffron.com.pk/
📄 Scraping: http://www.saffron.com.pk/

✅ Scraping complete!
   Emails found: 0
   Phones found: 0
   Social links: 0

📊 RESULTS
{
  "website": "http://www.saffron.com.pk/",
  "emails": [],
  "phones": [],
  "social_links": {
    "facebook": [],
    "instagram": [],
    "linkedin": [],
    "twitter": [],
    "youtube": []
  },
  "additional_pages": []
}

💾 Data saved to business_info.json


In [1]:
#!/usr/bin/env python3
"""
🌐 Universal Website Contact Info Crawler
- Starts from Google Maps link
- Scrapes all internal pages recursively
- Extracts emails, phones, social links, and addresses
- Requests first, Selenium fallback for JS-heavy pages
- Outputs JSON file
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup

# Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Webdriver manager
from webdriver_manager.chrome import ChromeDriverManager


class UniversalCrawler:
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    ADDRESS_PATTERN = re.compile(r'\d{1,5}\s\w+\s\w+|[A-Za-z0-9.,\s\-]+')  # Simplistic

    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }

    def __init__(self, headless: bool = True):
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def _init_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)

            wait = WebDriverWait(driver, 15)
            website_btn_selectors = [
                "//a[contains(@aria-label, 'Website')]",
                "//a[contains(text(),'Website')]"
            ]

            website_url = None
            for sel in website_btn_selectors:
                try:
                    element = wait.until(EC.presence_of_element_located((By.XPATH, sel)))
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # Remove Google redirect
                        if 'google.com/url?' in website_url:
                            from urllib.parse import unquote, parse_qs, urlparse
                            query = parse_qs(urlparse(website_url).query)
                            if 'q' in query:
                                website_url = unquote(query['q'][0])
                        break
                except TimeoutException:
                    continue

            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found on Google Maps listing")
                return None
        except Exception as e:
            print(f"❌ Error getting website: {e}")
            return None
        finally:
            if driver:
                driver.quit()

    def _extract_emails(self, text: str) -> Set[str]:
        return set(self.EMAIL_PATTERN.findall(text))

    def _extract_phones(self, text: str) -> Set[str]:
        numbers = self.PHONE_PATTERN.findall(text)
        cleaned = set()
        for p in numbers:
            clean = re.sub(r'\s+', '', p)
            if sum(c.isdigit() for c in clean) >= 9:
                cleaned.add(clean)
        return cleaned

    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        for a in soup.find_all('a', href=True):
            href = a['href'].lower()
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(d in href for d in domains):
                    full_url = urljoin(base_url, a['href'])
                    if full_url not in links[platform]:
                        links[platform].append(full_url)
        return links

    def _scrape_page(self, url: str) -> tuple:
        try:
            print(f"📄 Scraping: {url}")
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            for element in soup(['script', 'style', 'noscript']):
                element.decompose()

            text = soup.get_text()
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            social_links = self._extract_social_links(soup, url)

            return emails, phones, social_links, soup
        except Exception as e:
            print(f"⚠️ Requests failed for {url}, trying Selenium: {e}")
            # Fallback to Selenium
            try:
                driver = self._init_driver()
                driver.get(url)
                time.sleep(2)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                driver.quit()
                for element in soup(['script', 'style', 'noscript']):
                    element.decompose()
                text = soup.get_text()
                emails = self._extract_emails(text)
                phones = self._extract_phones(text)
                social_links = self._extract_social_links(soup, url)
                return emails, phones, social_links, soup
            except Exception as e2:
                print(f"❌ Selenium failed for {url}: {e2}")
                return set(), set(), {p: [] for p in self.SOCIAL_PLATFORMS}, None

    def _find_internal_links(self, soup: BeautifulSoup, base_url: str) -> List[str]:
        internal_links = []
        parsed_base = urlparse(base_url)
        for a in soup.find_all('a', href=True):
            href = a['href']
            parsed_href = urlparse(urljoin(base_url, href))
            if parsed_href.netloc == parsed_base.netloc:
                full_url = parsed_href.geturl()
                if full_url not in internal_links:
                    internal_links.append(full_url)
        return internal_links

    def crawl_website(self, base_url: str, max_pages: int = 80) -> Dict:
        print(f"\n🌐 Begin crawl for: {base_url}")
        visited = set()
        to_visit = [base_url]

        all_emails = set()
        all_phones = set()
        all_social = {platform: [] for platform in self.SOCIAL_PLATFORMS}

        page_count = 0
        while to_visit and page_count < max_pages:
            url = to_visit.pop(0)
            if url in visited:
                continue
            visited.add(url)
            page_count += 1

            emails, phones, social_links, soup = self._scrape_page(url)
            all_emails.update(emails)
            all_phones.update(phones)
            for p in all_social:
                all_social[p].extend([l for l in social_links[p] if l not in all_social[p]])

            if soup:
                internal_links = self._find_internal_links(soup, base_url)
                for link in internal_links:
                    if link not in visited and link not in to_visit:
                        to_visit.append(link)

            time.sleep(1)  # Polite delay

        return {
            'website': base_url,
            'emails': sorted(list(all_emails)),
            'phones': sorted(list(all_phones)),
            'social_links': {k: list(set(v)) for k, v in all_social.items()},
            'visited_pages': list(visited)
        }

    def scrape_from_gmaps(self, gmaps_url: str) -> Optional[Dict]:
        website = self.get_website_from_google_maps(gmaps_url)
        if website:
            return self.crawl_website(website)
        return None

    def save_to_json(self, data: Dict, filename: str = 'business_info.json'):
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f"💾 Data saved to {filename}")


def main():
    print("=" * 70)
    print("🗺️  WEBSITE CRAWLER FOR CONTACT INFO (requests + Selenium fallback)")
    print("=" * 70)

    gmaps_url = input("📍 Enter Google Maps URL: ").strip()
    if not gmaps_url:
        print("❌ No URL provided. Exiting.")
        return

    crawler = UniversalCrawler(headless=True)
    result = crawler.scrape_from_gmaps(gmaps_url)

    if result:
        print(json.dumps(result, indent=2, ensure_ascii=False))
        crawler.save_to_json(result)
    else:
        print("❌ Could not retrieve website or scrape data.")


if __name__ == "__main__":
    main()


🗺️  WEBSITE CRAWLER FOR CONTACT INFO (requests + Selenium fallback)
🔍 Opening Google Maps: https://www.google.com/maps/place/Rawalpindi+Medicare+Hospital/@33.6334588,73.0342839,14z/data=!4m6!3m5!1s0x38dfebade4e83c15:0x8072321813277499!8m2!3d33.6307011!4d73.0881662!16s%2Fg%2F11rs9hm8v1?authuser=0&entry=ttu&g_ep=EgoyMDI1MTEwNC4xIKXMDSoASAFQAw%3D%3D
✅ Found website: http://www.rmh.com.pk/

🌐 Begin crawl for: http://www.rmh.com.pk/
📄 Scraping: http://www.rmh.com.pk/
📄 Scraping: http://www.rmh.com.pk/#content
{
  "website": "http://www.rmh.com.pk/",
  "emails": [
    "info@rmh.com.pk"
  ],
  "phones": [
    "+923345933391"
  ],
  "social_links": {
    "facebook": [
      "http://web.archive.org/web/20230524195052/https://www.facebook.com/RastgarMaterialsHandling/"
    ],
    "instagram": [],
    "linkedin": [
      "http://web.archive.org/web/20230524195052/https://www.linkedin.com/company/rastgarmaterialshandling/"
    ],
    "twitter": [
      "https://pmx.com.pk/"
    ],
    "youtube": [

In [3]:
#!/usr/bin/env python3
"""
🌐 Focused Contact Info Scraper
- Gets main website from Google Maps
- Scrapes main page & priority pages only (contact/about)
- Extracts emails, phones, addresses, social links
- Output JSON contains only real contact info, main website, social media
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

class FocusedCrawler:
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    ADDRESS_KEYWORDS = ['street', 'road', 'plaza', 'building', 'suite', 'islamabad', 'lahore', 'karachi']

    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }

    PRIORITY_KEYWORDS = ['contact', 'about', 'reach', 'support', 'office']

    def __init__(self, headless: bool = True):
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def _init_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            wait = WebDriverWait(driver, 15)
            selectors = ["//a[contains(@aria-label, 'Website')]", "//a[contains(text(),'Website')]"]
            website_url = None
            for sel in selectors:
                try:
                    element = wait.until(EC.presence_of_element_located((By.XPATH, sel)))
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # remove Google redirect
                        from urllib.parse import unquote, parse_qs, urlparse
                        if 'google.com/url?' in website_url:
                            query = parse_qs(urlparse(website_url).query)
                            if 'q' in query:
                                website_url = unquote(query['q'][0])
                        break
                except TimeoutException:
                    continue
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found")
                return None
        finally:
            if driver:
                driver.quit()

    def _extract_emails(self, text: str) -> Set[str]:
        return set(self.EMAIL_PATTERN.findall(text))

    def _extract_phones(self, text: str) -> Set[str]:
        numbers = self.PHONE_PATTERN.findall(text)
        cleaned = set()
        for p in numbers:
            clean = re.sub(r'\s+', '', p)
            if sum(c.isdigit() for c in clean) >= 9:
                cleaned.add(clean)
        return cleaned

    def _extract_addresses(self, text: str) -> Set[str]:
        addresses = set()
        for line in text.splitlines():
            for keyword in self.ADDRESS_KEYWORDS:
                if keyword.lower() in line.lower() and len(line) > 10:
                    addresses.add(line.strip())
        return addresses

    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        for a in soup.find_all('a', href=True):
            href = a['href'].lower()
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(d in href for d in domains):
                    full_url = urljoin(base_url, a['href'])
                    if full_url not in links[platform]:
                        links[platform].append(full_url)
        return links

    def _scrape_page(self, url: str) -> tuple:
        try:
            print(f"📄 Scraping: {url}")
            resp = self.session.get(url, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            for el in soup(['script','style','noscript']):
                el.decompose()
            text = soup.get_text()
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            addresses = self._extract_addresses(text)
            social_links = self._extract_social_links(soup, url)
            return emails, phones, addresses, social_links
        except Exception as e:
            print(f"⚠️ Requests failed for {url}: {e}")
            return set(), set(), set(), {p: [] for p in self.SOCIAL_PLATFORMS}

    def scrape_main_website(self, website_url: str) -> Dict:
        urls_to_scrape = [website_url]  # main page
        main_emails, main_phones, main_addresses = set(), set(), set()
        main_social = {p: [] for p in self.SOCIAL_PLATFORMS}

        # Try priority pages first: contact/about
        try:
            resp = self.session.get(website_url, timeout=10)
            soup = BeautifulSoup(resp.text, 'html.parser')
            for a in soup.find_all('a', href=True):
                href = a['href']
                if any(k in href.lower() for k in self.PRIORITY_KEYWORDS):
                    full_url = urljoin(website_url, href)
                    if full_url not in urls_to_scrape:
                        urls_to_scrape.append(full_url)
        except:
            pass

        for url in urls_to_scrape:
            emails, phones, addresses, social_links = self._scrape_page(url)
            main_emails.update(emails)
            main_phones.update(phones)
            main_addresses.update(addresses)
            for p in main_social:
                main_social[p].extend([l for l in social_links[p] if l not in main_social[p]])

        return {
            "website": website_url,
            "emails": sorted(list(main_emails)),
            "phones": sorted(list(main_phones)),
            "addresses": sorted(list(main_addresses)),
            "social_links": {p: list(set(v)) for p,v in main_social.items()}
        }

    def scrape_from_gmaps(self, gmaps_url: str) -> Optional[Dict]:
        website = self.get_website_from_google_maps(gmaps_url)
        if website:
            return self.scrape_main_website(website)
        return None

    def save_to_json(self, data: Dict, filename='business_info.json'):
        with open(filename,'w',encoding='utf-8') as f:
            json.dump(data,f,indent=2,ensure_ascii=False)
        print(f"💾 Data saved to {filename}")

def main():
    gmaps_url = input("📍 Enter Google Maps URL: ").strip()
    if not gmaps_url:
        print("❌ No URL provided")
        return
    crawler = FocusedCrawler(headless=True)
    result = crawler.scrape_from_gmaps(gmaps_url)
    if result:
        print(json.dumps(result, indent=2, ensure_ascii=False))
        crawler.save_to_json(result)
    else:
        print("❌ Could not scrape website")

if __name__ == "__main__":
    main()


🔍 Opening Google Maps: https://www.google.com/maps/place/Rawalpindi+Medicare+Hospital/@33.6334588,73.0342839,14z/data=!4m6!3m5!1s0x38dfebade4e83c15:0x8072321813277499!8m2!3d33.6307011!4d73.0881662!16s%2Fg%2F11rs9hm8v1?authuser=0&entry=ttu&g_ep=EgoyMDI1MTEwNC4xIKXMDSoASAFQAw%3D%3D
✅ Found website: http://www.rmh.com.pk/
📄 Scraping: http://www.rmh.com.pk/
📄 Scraping: https://rmh.com.pk/about-us/
📄 Scraping: https://rmh.com.pk/contact-us/
📄 Scraping: https://rmh.com.pk/reach-trucks/
{
  "website": "http://www.rmh.com.pk/",
  "emails": [
    "info@rmh.com.pk"
  ],
  "phones": [
    "+922135123112",
    "+922135123114",
    "+922135123116-17",
    "+923345933391",
    "+923444437099"
  ],
  "addresses": [
    "Darul Uloom Plot # 9, Shahrah-e-Darul Uloom, Sector 28, Korangi Industrial Area, Karachi – 75180, Pakistan",
    "House # 344, St # 69, F-11/1, Nazim-ud-din Road Islamabad",
    "Islamabad Office",
    "Lahore Office",
    "Plot No. 123, Sundar Industrial Area, Gate No.2, Lahore Pakis

In [4]:
#!/usr/bin/env python3
"""
🌐 Focused Contact Info Scraper
- Gets main website from Google Maps
- Scrapes main page & priority pages only (contact/about)
- Extracts emails, phones, addresses, social links
- Output JSON contains only real contact info, main website, social media
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

class FocusedCrawler:
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    ADDRESS_KEYWORDS = ['street', 'road', 'plaza', 'building', 'suite', 'islamabad', 'lahore', 'karachi']

    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }

    PRIORITY_KEYWORDS = ['contact', 'about', 'reach', 'support', 'office']

    def __init__(self, headless: bool = True):
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def _init_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            wait = WebDriverWait(driver, 15)
            selectors = ["//a[contains(@aria-label, 'Website')]", "//a[contains(text(),'Website')]"]
            website_url = None
            for sel in selectors:
                try:
                    element = wait.until(EC.presence_of_element_located((By.XPATH, sel)))
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # remove Google redirect
                        from urllib.parse import unquote, parse_qs, urlparse
                        if 'google.com/url?' in website_url:
                            query = parse_qs(urlparse(website_url).query)
                            if 'q' in query:
                                website_url = unquote(query['q'][0])
                        break
                except TimeoutException:
                    continue
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found")
                return None
        finally:
            if driver:
                driver.quit()

    def _extract_emails(self, text: str) -> Set[str]:
        return set(self.EMAIL_PATTERN.findall(text))

    def _extract_phones(self, text: str) -> Set[str]:
        numbers = self.PHONE_PATTERN.findall(text)
        cleaned = set()
        for p in numbers:
            clean = re.sub(r'\s+', '', p)
            if sum(c.isdigit() for c in clean) >= 9:
                cleaned.add(clean)
        return cleaned

    def _extract_addresses(self, text: str) -> Set[str]:
        addresses = set()
        for line in text.splitlines():
            for keyword in self.ADDRESS_KEYWORDS:
                if keyword.lower() in line.lower() and len(line) > 10:
                    addresses.add(line.strip())
        return addresses

    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        for a in soup.find_all('a', href=True):
            href = a['href'].lower()
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(d in href for d in domains):
                    full_url = urljoin(base_url, a['href'])
                    if full_url not in links[platform]:
                        links[platform].append(full_url)
        return links

    def _scrape_page(self, url: str) -> tuple:
        try:
            print(f"📄 Scraping: {url}")
            resp = self.session.get(url, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            for el in soup(['script','style','noscript']):
                el.decompose()
            text = soup.get_text()
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            addresses = self._extract_addresses(text)
            social_links = self._extract_social_links(soup, url)
            return emails, phones, addresses, social_links
        except Exception as e:
            print(f"⚠️ Requests failed for {url}: {e}")
            return set(), set(), set(), {p: [] for p in self.SOCIAL_PLATFORMS}

    def scrape_main_website(self, website_url: str) -> Dict:
        urls_to_scrape = [website_url]  # main page
        main_emails, main_phones, main_addresses = set(), set(), set()
        main_social = {p: [] for p in self.SOCIAL_PLATFORMS}

        # Try priority pages first: contact/about
        try:
            resp = self.session.get(website_url, timeout=10)
            soup = BeautifulSoup(resp.text, 'html.parser')
            for a in soup.find_all('a', href=True):
                href = a['href']
                if any(k in href.lower() for k in self.PRIORITY_KEYWORDS):
                    full_url = urljoin(website_url, href)
                    if full_url not in urls_to_scrape:
                        urls_to_scrape.append(full_url)
        except:
            pass

        for url in urls_to_scrape:
            emails, phones, addresses, social_links = self._scrape_page(url)
            main_emails.update(emails)
            main_phones.update(phones)
            main_addresses.update(addresses)
            for p in main_social:
                main_social[p].extend([l for l in social_links[p] if l not in main_social[p]])

        return {
            "website": website_url,
            "emails": sorted(list(main_emails)),
            "phones": sorted(list(main_phones)),
            "addresses": sorted(list(main_addresses)),
            "social_links": {p: list(set(v)) for p,v in main_social.items()}
        }

    def scrape_from_gmaps(self, gmaps_url: str) -> Optional[Dict]:
        website = self.get_website_from_google_maps(gmaps_url)
        if website:
            return self.scrape_main_website(website)
        return None

    def save_to_json(self, data: Dict, filename='business_info.json'):
        with open(filename,'w',encoding='utf-8') as f:
            json.dump(data,f,indent=2,ensure_ascii=False)
        print(f"💾 Data saved to {filename}")

def main():
    gmaps_url = input("📍 Enter Google Maps URL: ").strip()
    if not gmaps_url:
        print("❌ No URL provided")
        return
    crawler = FocusedCrawler(headless=True)
    result = crawler.scrape_from_gmaps(gmaps_url)
    if result:
        print(json.dumps(result, indent=2, ensure_ascii=False))
        crawler.save_to_json(result)
    else:
        print("❌ Could not scrape website")

if __name__ == "__main__":
    main()


🔍 Opening Google Maps: https://www.google.com/maps/place/NYC+Health+%2B+Hospitals+%2F+Bellevue/@40.7396865,-74.2645955,11z/data=!3m1!5s0x89c2599e8c7ec08d:0x5caf69bb05ed42d1!4m6!3m5!1s0x89c2590c212fc7a9:0x67a293dafcc5f7e0!8m2!3d40.7396865!4d-73.9762044!16zL20vMDYwMGp0?authuser=0&entry=ttu&g_ep=EgoyMDI1MTEwNC4xIKXMDSoASAFQAw%3D%3D
✅ Found website: http://www.nychealthandhospitals.org/bellevue/
📄 Scraping: http://www.nychealthandhospitals.org/bellevue/
{
  "website": "http://www.nychealthandhospitals.org/bellevue/",
  "emails": [],
  "phones": [],
  "addresses": [],
  "social_links": {
    "facebook": [],
    "instagram": [],
    "linkedin": [],
    "twitter": [],
    "youtube": []
  }
}
💾 Data saved to business_info.json


In [11]:
#!/usr/bin/env python3
"""
🌐 Focused Contact Info Scraper
- Gets main website from Google Maps
- Scrapes main page & priority pages only (contact/about)
- Extracts emails, phones, addresses, social links
- Output JSON contains only real contact info, main website, social media
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

class FocusedCrawler:
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    ADDRESS_KEYWORDS = ['street', 'road', 'plaza', 'building', 'suite', 'islamabad', 'lahore', 'karachi']

    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }

    PRIORITY_KEYWORDS = ['contact', 'about', 'reach', 'support', 'office']

    def __init__(self, headless: bool = True):
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def _init_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            wait = WebDriverWait(driver, 15)
            selectors = ["//a[contains(@aria-label, 'Website')]", "//a[contains(text(),'Website')]"]
            website_url = None
            for sel in selectors:
                try:
                    element = wait.until(EC.presence_of_element_located((By.XPATH, sel)))
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # remove Google redirect
                        from urllib.parse import unquote, parse_qs, urlparse
                        if 'google.com/url?' in website_url:
                            query = parse_qs(urlparse(website_url).query)
                            if 'q' in query:
                                website_url = unquote(query['q'][0])
                        break
                except TimeoutException:
                    continue
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found")
                return None
        finally:
            if driver:
                driver.quit()

    def _extract_emails(self, text: str) -> Set[str]:
        return set(self.EMAIL_PATTERN.findall(text))

    def _extract_phones(self, text: str) -> Set[str]:
        numbers = self.PHONE_PATTERN.findall(text)
        cleaned = set()
        for p in numbers:
            clean = re.sub(r'\s+', '', p)
            if sum(c.isdigit() for c in clean) >= 9:
                cleaned.add(clean)
        return cleaned

    def _extract_addresses(self, text: str) -> Set[str]:
        addresses = set()
        for line in text.splitlines():
            for keyword in self.ADDRESS_KEYWORDS:
                if keyword.lower() in line.lower() and len(line) > 10:
                    addresses.add(line.strip())
        return addresses

    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        for a in soup.find_all('a', href=True):
            href = a['href'].lower()
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(d in href for d in domains):
                    full_url = urljoin(base_url, a['href'])
                    if full_url not in links[platform]:
                        links[platform].append(full_url)
        return links

    def _scrape_page(self, url: str) -> tuple:
        try:
            print(f"📄 Scraping: {url}")
            resp = self.session.get(url, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            for el in soup(['script','style','noscript']):
                el.decompose()
            text = soup.get_text()
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            addresses = self._extract_addresses(text)
            social_links = self._extract_social_links(soup, url)
            return emails, phones, addresses, social_links
        except Exception as e:
            print(f"⚠️ Requests failed for {url}: {e}")
            return set(), set(), set(), {p: [] for p in self.SOCIAL_PLATFORMS}

    def scrape_main_website(self, website_url: str) -> Dict:
        urls_to_scrape = [website_url]  # main page
        main_emails, main_phones, main_addresses = set(), set(), set()
        main_social = {p: [] for p in self.SOCIAL_PLATFORMS}

        # Try priority pages first: contact/about
        try:
            resp = self.session.get(website_url, timeout=10)
            soup = BeautifulSoup(resp.text, 'html.parser')
            for a in soup.find_all('a', href=True):
                href = a['href']
                if any(k in href.lower() for k in self.PRIORITY_KEYWORDS):
                    full_url = urljoin(website_url, href)
                    if full_url not in urls_to_scrape:
                        urls_to_scrape.append(full_url)
        except:
            pass

        for url in urls_to_scrape:
            emails, phones, addresses, social_links = self._scrape_page(url)
            main_emails.update(emails)
            main_phones.update(phones)
            main_addresses.update(addresses)
            for p in main_social:
                main_social[p].extend([l for l in social_links[p] if l not in main_social[p]])

        return {
            "website": website_url,
            "emails": sorted(list(main_emails)),
            "phones": sorted(list(main_phones)),
            "addresses": sorted(list(main_addresses)),
            "social_links": {p: list(set(v)) for p,v in main_social.items()}
        }

    def scrape_from_gmaps(self, gmaps_url: str) -> Optional[Dict]:
        website = self.get_website_from_google_maps(gmaps_url)
        if website:
            return self.scrape_main_website(website)
        return None

    def save_to_json(self, data: Dict, filename='business_info.json'):
        with open(filename,'w',encoding='utf-8') as f:
            json.dump(data,f,indent=2,ensure_ascii=False)
        print(f"💾 Data saved to {filename}")

def main():
    gmaps_url = input("📍 Enter Google Maps URL: ").strip()
    if not gmaps_url:
        print("❌ No URL provided")
        return
    crawler = FocusedCrawler(headless=True)
    result = crawler.scrape_from_gmaps(gmaps_url)
    if result:
        print(json.dumps(result, indent=2, ensure_ascii=False))
        crawler.save_to_json(result)
    else:
        print("❌ Could not scrape website")

if __name__ == "__main__":
    main()


====== WebDriver manager ======


🔍 Opening Google Maps: https://www.google.com/maps/place/The+Mount+Sinai+Hospital/@40.7396865,-74.2645955,11z/data=!3m1!5s0x89c2377a56a7853f:0x4ad2a9bc5cffd767!4m6!3m5!1s0x89c2f63dcaeeda93:0x9797c11e6d7bc63f!8m2!3d40.7899484!4d-73.9524454!16zL20vMDRsZjNi?authuser=0&entry=ttu&g_ep=EgoyMDI1MTEwNC4xIKXMDSoASAFQAw%3D%3D


Get LATEST chromedriver version for google-chrome
Get LATEST chromedriver version for google-chrome
Driver [C:\Users\ABC\.wdm\drivers\chromedriver\win64\142.0.7444.61\chromedriver-win32/chromedriver.exe] found in cache


✅ Found website: https://www.mountsinai.org/locations/mount-sinai?utm_source=Yext&utm_medium=local_listing&utm_campaign=MountSinai&y_source=1_MTgwODM1MS03MTUtbG9jYXRpb24ud2Vic2l0ZQ==
📄 Scraping: https://www.mountsinai.org/locations/mount-sinai?utm_source=Yext&utm_medium=local_listing&utm_campaign=MountSinai&y_source=1_MTgwODM1MS03MTUtbG9jYXRpb24ud2Vic2l0ZQ==
📄 Scraping: https://www.mountsinai.org/about/patient-tools-and-resources/check-symptoms-get-care
📄 Scraping: https://www.mountsinai.org/about/access
📄 Scraping: https://www.mountsinai.org/about/network
📄 Scraping: https://www.mountsinai.org/about
📄 Scraping: https://www.mountsinai.org/about/ethics
📄 Scraping: https://www.mountsinai.org/about/contact
📄 Scraping: https://www.mountsinai.org/about/patient-tools-and-resources
📄 Scraping: https://www.mountsinai.org/about/executive-leadership
📄 Scraping: https://www.mountsinai.org/about/facts
📄 Scraping: https://www.mountsinai.org/about/financial-assistance
📄 Scraping: https://www.mountsi

In [13]:
#!/usr/bin/env python3
"""
🌐 Focused Contact Info Scraper
- Gets main website from Google Maps
- Scrapes main page & priority pages only (contact/about)
- Extracts emails, phones, addresses, social links
- Output JSON contains only real contact info, main website, social media
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

class FocusedCrawler:
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    ADDRESS_KEYWORDS = ['street', 'road', 'plaza', 'building', 'suite', 'islamabad', 'lahore', 'karachi']

    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }

    PRIORITY_KEYWORDS = ['contact', 'about', 'reach', 'support', 'office']

    def __init__(self, headless: bool = True):
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def _init_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            wait = WebDriverWait(driver, 15)
            selectors = ["//a[contains(@aria-label, 'Website')]", "//a[contains(text(),'Website')]"]
            website_url = None
            for sel in selectors:
                try:
                    element = wait.until(EC.presence_of_element_located((By.XPATH, sel)))
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # remove Google redirect
                        from urllib.parse import unquote, parse_qs, urlparse
                        if 'google.com/url?' in website_url:
                            query = parse_qs(urlparse(website_url).query)
                            if 'q' in query:
                                website_url = unquote(query['q'][0])
                        break
                except TimeoutException:
                    continue
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found")
                return None
        finally:
            if driver:
                driver.quit()

    def _extract_emails(self, text: str) -> Set[str]:
        return set(self.EMAIL_PATTERN.findall(text))

    def _extract_phones(self, text: str) -> Set[str]:
        numbers = self.PHONE_PATTERN.findall(text)
        cleaned = set()
        for p in numbers:
            clean = re.sub(r'\s+', '', p)
            if sum(c.isdigit() for c in clean) >= 9:
                cleaned.add(clean)
        return cleaned

    def _extract_addresses(self, text: str) -> Set[str]:
        addresses = set()
        for line in text.splitlines():
            for keyword in self.ADDRESS_KEYWORDS:
                if keyword.lower() in line.lower() and len(line) > 10:
                    addresses.add(line.strip())
        return addresses

    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        for a in soup.find_all('a', href=True):
            href = a['href'].lower()
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(d in href for d in domains):
                    full_url = urljoin(base_url, a['href'])
                    if full_url not in links[platform]:
                        links[platform].append(full_url)
        return links

    def _scrape_page(self, url: str) -> tuple:
        try:
            print(f"📄 Scraping: {url}")
            resp = self.session.get(url, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            for el in soup(['script','style','noscript']):
                el.decompose()
            text = soup.get_text()
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            addresses = self._extract_addresses(text)
            social_links = self._extract_social_links(soup, url)
            return emails, phones, addresses, social_links
        except Exception as e:
            print(f"⚠️ Requests failed for {url}: {e}")
            return set(), set(), set(), {p: [] for p in self.SOCIAL_PLATFORMS}

    def scrape_main_website(self, website_url: str) -> Dict:
        urls_to_scrape = [website_url]  # main page
        main_emails, main_phones, main_addresses = set(), set(), set()
        main_social = {p: [] for p in self.SOCIAL_PLATFORMS}

        # Try priority pages first: contact/about
        try:
            resp = self.session.get(website_url, timeout=10)
            soup = BeautifulSoup(resp.text, 'html.parser')
            for a in soup.find_all('a', href=True):
                href = a['href']
                if any(k in href.lower() for k in self.PRIORITY_KEYWORDS):
                    full_url = urljoin(website_url, href)
                    if full_url not in urls_to_scrape:
                        urls_to_scrape.append(full_url)
        except:
            pass

        for url in urls_to_scrape:
            emails, phones, addresses, social_links = self._scrape_page(url)
            main_emails.update(emails)
            main_phones.update(phones)
            main_addresses.update(addresses)
            for p in main_social:
                main_social[p].extend([l for l in social_links[p] if l not in main_social[p]])

        return {
            "website": website_url,
            "emails": sorted(list(main_emails)),
            "phones": sorted(list(main_phones)),
            "addresses": sorted(list(main_addresses)),
            "social_links": {p: list(set(v)) for p,v in main_social.items()}
        }

    def scrape_from_gmaps(self, gmaps_url: str) -> Optional[Dict]:
        website = self.get_website_from_google_maps(gmaps_url)
        if website:
            return self.scrape_main_website(website)
        return None

    def save_to_json(self, data: Dict, filename='business_info.json'):
        with open(filename,'w',encoding='utf-8') as f:
            json.dump(data,f,indent=2,ensure_ascii=False)
        print(f"💾 Data saved to {filename}")

def main():
    gmaps_url = input("📍 Enter Google Maps URL: ").strip()
    if not gmaps_url:
        print("❌ No URL provided")
        return
    crawler = FocusedCrawler(headless=True)
    result = crawler.scrape_from_gmaps(gmaps_url)
    if result:
        print(json.dumps(result, indent=2, ensure_ascii=False))
        crawler.save_to_json(result)
    else:
        print("❌ Could not scrape website")

if __name__ == "__main__":
    main()


====== WebDriver manager ======


🔍 Opening Google Maps: https://maps.app.goo.gl/izAmtdEixL4P9TBo6


Get LATEST chromedriver version for google-chrome
Get LATEST chromedriver version for google-chrome
Driver [C:\Users\ABC\.wdm\drivers\chromedriver\win64\142.0.7444.61\chromedriver-win32/chromedriver.exe] found in cache


✅ Found website: http://www.nyp.org/
📄 Scraping: http://www.nyp.org/
📄 Scraping: http://www.nyp.org/about
📄 Scraping: http://www.nyp.org/contact
📄 Scraping: https://healthmatters.nyp.org/what-to-know-about-the-new-covid-variant-nb-1-8-1-known-as-nimbus-and-how-to-protect-yourself-this-summer/
📄 Scraping: http://www.nyp.org/about/community
{
  "website": "http://www.nyp.org/",
  "emails": [],
  "phones": [
    "10032212-305-2500",
    "10034212-932-4000",
    "10038212-312-5000",
    "10065212-746-5454",
    "10567914-737-9000",
    "10605914-682-9100",
    "10708914-787-1000",
    "11355718-670-2000",
    "212-305-2500",
    "212-305-5437",
    "212-312-5000",
    "212-543-8862",
    "212-746-5454",
    "212-932-4000",
    "718-499-2273",
    "718-670-2000",
    "718-780-3000",
    "718-780-5367",
    "800-245-5437",
    "800-282-6684",
    "855-969-7564",
    "866-697-2553",
    "877-697-9355",
    "888-694-5700",
    "888-697-4010",
    "914-682-9100",
    "914-737-9000",
    "914-78

In [16]:
#!/usr/bin/env python3
"""
🌐 Focused Contact Info Scraper
- Gets main website from Google Maps
- Scrapes main page & priority pages only (contact/about)
- Extracts emails, phones, addresses, social links
- Output JSON contains only real contact info, main website, social media
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

class FocusedCrawler:
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    ADDRESS_KEYWORDS = ['street', 'road', 'plaza', 'building', 'suite', 'islamabad', 'lahore', 'karachi']

    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }

    PRIORITY_KEYWORDS = ['contact', 'about', 'reach', 'support', 'office']

    def __init__(self, headless: bool = True):
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def _init_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            wait = WebDriverWait(driver, 15)
            selectors = ["//a[contains(@aria-label, 'Website')]", "//a[contains(text(),'Website')]"]
            website_url = None
            for sel in selectors:
                try:
                    element = wait.until(EC.presence_of_element_located((By.XPATH, sel)))
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # remove Google redirect
                        from urllib.parse import unquote, parse_qs, urlparse
                        if 'google.com/url?' in website_url:
                            query = parse_qs(urlparse(website_url).query)
                            if 'q' in query:
                                website_url = unquote(query['q'][0])
                        break
                except TimeoutException:
                    continue
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found")
                return None
        finally:
            if driver:
                driver.quit()

    def _extract_emails(self, text: str) -> Set[str]:
        return set(self.EMAIL_PATTERN.findall(text))

    def _extract_phones(self, text: str) -> Set[str]:
        numbers = self.PHONE_PATTERN.findall(text)
        cleaned = set()
        for p in numbers:
            clean = re.sub(r'\s+', '', p)
            if sum(c.isdigit() for c in clean) >= 9:
                cleaned.add(clean)
        return cleaned

    def _extract_addresses(self, text: str) -> Set[str]:
        addresses = set()
        for line in text.splitlines():
            for keyword in self.ADDRESS_KEYWORDS:
                if keyword.lower() in line.lower() and len(line) > 10:
                    addresses.add(line.strip())
        return addresses

    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        for a in soup.find_all('a', href=True):
            href = a['href'].lower()
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(d in href for d in domains):
                    full_url = urljoin(base_url, a['href'])
                    if full_url not in links[platform]:
                        links[platform].append(full_url)
        return links

    def _scrape_page(self, url: str) -> tuple:
        try:
            print(f"📄 Scraping: {url}")
            resp = self.session.get(url, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            for el in soup(['script','style','noscript']):
                el.decompose()
            text = soup.get_text()
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            addresses = self._extract_addresses(text)
            social_links = self._extract_social_links(soup, url)
            return emails, phones, addresses, social_links
        except Exception as e:
            print(f"⚠️ Requests failed for {url}: {e}")
            return set(), set(), set(), {p: [] for p in self.SOCIAL_PLATFORMS}

    def scrape_main_website(self, website_url: str) -> Dict:
        urls_to_scrape = [website_url]  # main page
        main_emails, main_phones, main_addresses = set(), set(), set()
        main_social = {p: [] for p in self.SOCIAL_PLATFORMS}

        # Try priority pages first: contact/about
        try:
            resp = self.session.get(website_url, timeout=10)
            soup = BeautifulSoup(resp.text, 'html.parser')
            for a in soup.find_all('a', href=True):
                href = a['href']
                if any(k in href.lower() for k in self.PRIORITY_KEYWORDS):
                    full_url = urljoin(website_url, href)
                    if full_url not in urls_to_scrape:
                        urls_to_scrape.append(full_url)
        except:
            pass

        for url in urls_to_scrape:
            emails, phones, addresses, social_links = self._scrape_page(url)
            main_emails.update(emails)
            main_phones.update(phones)
            main_addresses.update(addresses)
            for p in main_social:
                main_social[p].extend([l for l in social_links[p] if l not in main_social[p]])

        return {
            "website": website_url,
            "emails": sorted(list(main_emails)),
            "phones": sorted(list(main_phones)),
            "addresses": sorted(list(main_addresses)),
            "social_links": {p: list(set(v)) for p,v in main_social.items()}
        }

    def scrape_from_gmaps(self, gmaps_url: str) -> Optional[Dict]:
        website = self.get_website_from_google_maps(gmaps_url)
        if website:
            return self.scrape_main_website(website)
        return None

    def save_to_json(self, data: Dict, filename='business_info1.json'):
        with open(filename,'w',encoding='utf-8') as f:
            json.dump(data,f,indent=2,ensure_ascii=False)
        print(f"💾 Data saved to {filename}")

def main():
    gmaps_url = input("📍 Enter Google Maps URL: ").strip()
    if not gmaps_url:
        print("❌ No URL provided")
        return
    crawler = FocusedCrawler(headless=True)
    result = crawler.scrape_from_gmaps(gmaps_url)
    if result:
        print(json.dumps(result, indent=2, ensure_ascii=False))
        crawler.save_to_json(result)
    else:
        print("❌ Could not scrape website")

if __name__ == "__main__":
    main()


====== WebDriver manager ======


🔍 Opening Google Maps: https://maps.app.goo.gl/Rj2r8F4QMRCfph9L9


Get LATEST chromedriver version for google-chrome
Get LATEST chromedriver version for google-chrome
Driver [C:\Users\ABC\.wdm\drivers\chromedriver\win64\142.0.7444.61\chromedriver-win32/chromedriver.exe] found in cache


✅ Found website: http://www.vu.edu.pk/
📄 Scraping: http://www.vu.edu.pk/
📄 Scraping: http://www.vu.edu.pk/AboutUs/AboutVU.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/visionAndmission.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/Education.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/Milestones.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/MOUsCollaboration.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/organization.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/BOG.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/Directorates.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/Faculty.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/AdjunctFaculty.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/RTI.aspx
📄 Scraping: http://www.vu.edu.pk/AboutUs/ProgramsOffered.aspx?Degree=BS-Lateral
📄 Scraping: http://www.vu.edu.pk/AboutUs/ProgramsOffered.aspx?Degree=BS
📄 Scraping: http://www.vu.edu.pk/AboutUs/ProgramsOffered.aspx?Degree=B.Ed
📄 Scraping: http://www.vu.edu.pk/AboutUs/ProgramsOffered.aspx?Degree=Assoc

In [1]:
#!/usr/bin/env python3
"""
🌐 Focused Contact Info Scraper
- Gets main website from Google Maps
- Scrapes main page & priority pages only (contact/about)
- Extracts emails, phones, addresses, social links
- Output JSON contains only real contact info, main website, social media
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

class FocusedCrawler:
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    ADDRESS_KEYWORDS = ['street', 'road', 'plaza', 'building', 'suite', 'islamabad', 'lahore', 'karachi']

    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }

    PRIORITY_KEYWORDS = ['contact', 'about', 'reach', 'support', 'office']

    def __init__(self, headless: bool = True):
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def _init_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            wait = WebDriverWait(driver, 15)
            selectors = ["//a[contains(@aria-label, 'Website')]", "//a[contains(text(),'Website')]"]
            website_url = None
            for sel in selectors:
                try:
                    element = wait.until(EC.presence_of_element_located((By.XPATH, sel)))
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # remove Google redirect
                        from urllib.parse import unquote, parse_qs, urlparse
                        if 'google.com/url?' in website_url:
                            query = parse_qs(urlparse(website_url).query)
                            if 'q' in query:
                                website_url = unquote(query['q'][0])
                        break
                except TimeoutException:
                    continue
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found")
                return None
        finally:
            if driver:
                driver.quit()

    def _extract_emails(self, text: str) -> Set[str]:
        return set(self.EMAIL_PATTERN.findall(text))

    def _extract_phones(self, text: str) -> Set[str]:
        numbers = self.PHONE_PATTERN.findall(text)
        cleaned = set()
        for p in numbers:
            clean = re.sub(r'\s+', '', p)
            if sum(c.isdigit() for c in clean) >= 9:
                cleaned.add(clean)
        return cleaned

    def _extract_addresses(self, text: str) -> Set[str]:
        addresses = set()
        for line in text.splitlines():
            for keyword in self.ADDRESS_KEYWORDS:
                if keyword.lower() in line.lower() and len(line) > 10:
                    addresses.add(line.strip())
        return addresses

    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        for a in soup.find_all('a', href=True):
            href = a['href'].lower()
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(d in href for d in domains):
                    full_url = urljoin(base_url, a['href'])
                    if full_url not in links[platform]:
                        links[platform].append(full_url)
        return links

    def _scrape_page(self, url: str) -> tuple:
        try:
            print(f"📄 Scraping: {url}")
            resp = self.session.get(url, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            for el in soup(['script','style','noscript']):
                el.decompose()
            text = soup.get_text()
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            addresses = self._extract_addresses(text)
            social_links = self._extract_social_links(soup, url)
            return emails, phones, addresses, social_links
        except Exception as e:
            print(f"⚠️ Requests failed for {url}: {e}")
            return set(), set(), set(), {p: [] for p in self.SOCIAL_PLATFORMS}

    def scrape_main_website(self, website_url: str) -> Dict:
        urls_to_scrape = [website_url]  # main page
        main_emails, main_phones, main_addresses = set(), set(), set()
        main_social = {p: [] for p in self.SOCIAL_PLATFORMS}

        # Try priority pages first: contact/about
        try:
            resp = self.session.get(website_url, timeout=10)
            soup = BeautifulSoup(resp.text, 'html.parser')
            for a in soup.find_all('a', href=True):
                href = a['href']
                if any(k in href.lower() for k in self.PRIORITY_KEYWORDS):
                    full_url = urljoin(website_url, href)
                    if full_url not in urls_to_scrape:
                        urls_to_scrape.append(full_url)
        except:
            pass

        for url in urls_to_scrape:
            emails, phones, addresses, social_links = self._scrape_page(url)
            main_emails.update(emails)
            main_phones.update(phones)
            main_addresses.update(addresses)
            for p in main_social:
                main_social[p].extend([l for l in social_links[p] if l not in main_social[p]])

        return {
            "website": website_url,
            "emails": sorted(list(main_emails)),
            "phones": sorted(list(main_phones)),
            "addresses": sorted(list(main_addresses)),
            "social_links": {p: list(set(v)) for p,v in main_social.items()}
        }

    def scrape_from_gmaps(self, gmaps_url: str) -> Optional[Dict]:
        website = self.get_website_from_google_maps(gmaps_url)
        if website:
            return self.scrape_main_website(website)
        return None

    def save_to_json(self, data: Dict, filename='business_info3.json'):
        with open(filename,'w',encoding='utf-8') as f:
            json.dump(data,f,indent=2,ensure_ascii=False)
        print(f"💾 Data saved to {filename}")

def main():
    gmaps_url = input("📍 Enter Google Maps URL: ").strip()
    if not gmaps_url:
        print("❌ No URL provided")
        return
    crawler = FocusedCrawler(headless=True)
    result = crawler.scrape_from_gmaps(gmaps_url)
    if result:
        print(json.dumps(result, indent=2, ensure_ascii=False))
        crawler.save_to_json(result)
    else:
        print("❌ Could not scrape website")

if __name__ == "__main__":
    main()


🔍 Opening Google Maps: https://maps.app.goo.gl/CjSpy46fYEErzZpy7
✅ Found website: https://zainabclinics.com/
📄 Scraping: https://zainabclinics.com/
⚠️ Requests failed for https://zainabclinics.com/: HTTPSConnectionPool(host='zainabclinics.com', port=443): Read timed out.
📄 Scraping: https://zainabclinics.com/about-us-zainab-dental-clinic/
📄 Scraping: https://zainabclinics.com/contact-us-zainab-dental-clinic/
📄 Scraping: https://zainabclinics.com/contact-us/
📄 Scraping: https://zainabclinics.com/about-us/
{
  "website": "https://zainabclinics.com/",
  "emails": [
    "info@zainabclinics.com"
  ],
  "phones": [
    "03435485407"
  ],
  "addresses": [
    "Main PWD Rd, opposite bank alfalah, Block C Sector C PWD Society, Islamabad, 44000",
    "Your Trusted Dental Clinic in PWD, Islamabad",
    "Your Trusted Dental Health Partner in PWD, Islamabad"
  ],
  "social_links": {
    "facebook": [
      "https://www.facebook.com/ZainabMedical/"
    ],
    "instagram": [
      "https://www.instag

In [3]:
#!/usr/bin/env python3
"""
🌐 Focused Contact Info Scraper
- Gets main website from Google Maps
- Scrapes main page & priority pages only (contact/about)
- Extracts emails, phones, addresses, social links
- Output JSON contains only real contact info, main website, social media
"""

import re
import json
import time
from urllib.parse import urljoin, urlparse
from typing import Dict, List, Set, Optional

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

class FocusedCrawler:
    EMAIL_PATTERN = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}')
    PHONE_PATTERN = re.compile(r'\+?\d[\d\s\-]{8,}\d')
    ADDRESS_KEYWORDS = ['street', 'road', 'plaza', 'building', 'suite', 'islamabad', 'lahore', 'karachi']

    SOCIAL_PLATFORMS = {
        'facebook': ['facebook.com', 'fb.com'],
        'instagram': ['instagram.com'],
        'linkedin': ['linkedin.com'],
        'twitter': ['twitter.com', 'x.com'],
        'youtube': ['youtube.com', 'youtu.be']
    }

    PRIORITY_KEYWORDS = ['contact', 'about', 'reach', 'support', 'office']

    def __init__(self, headless: bool = True):
        self.headless = headless
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        })

    def _init_driver(self) -> webdriver.Chrome:
        options = Options()
        if self.headless:
            options.add_argument("--headless=new")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")

        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        return driver

    def get_website_from_google_maps(self, gmaps_url: str) -> Optional[str]:
        driver = None
        try:
            print(f"🔍 Opening Google Maps: {gmaps_url}")
            driver = self._init_driver()
            driver.get(gmaps_url)
            wait = WebDriverWait(driver, 15)
            selectors = ["//a[contains(@aria-label, 'Website')]", "//a[contains(text(),'Website')]"]
            website_url = None
            for sel in selectors:
                try:
                    element = wait.until(EC.presence_of_element_located((By.XPATH, sel)))
                    website_url = element.get_attribute('href')
                    if website_url and website_url.startswith('http'):
                        # remove Google redirect
                        from urllib.parse import unquote, parse_qs, urlparse
                        if 'google.com/url?' in website_url:
                            query = parse_qs(urlparse(website_url).query)
                            if 'q' in query:
                                website_url = unquote(query['q'][0])
                        break
                except TimeoutException:
                    continue
            if website_url:
                print(f"✅ Found website: {website_url}")
                return website_url
            else:
                print("❌ No website found")
                return None
        finally:
            if driver:
                driver.quit()

    def _extract_emails(self, text: str) -> Set[str]:
        return set(self.EMAIL_PATTERN.findall(text))

    def _extract_phones(self, text: str) -> Set[str]:
        numbers = self.PHONE_PATTERN.findall(text)
        cleaned = set()
        for p in numbers:
            clean = re.sub(r'\s+', '', p)
            if sum(c.isdigit() for c in clean) >= 9:
                cleaned.add(clean)
        return cleaned

    def _extract_addresses(self, text: str) -> Set[str]:
        addresses = set()
        for line in text.splitlines():
            for keyword in self.ADDRESS_KEYWORDS:
                if keyword.lower() in line.lower() and len(line) > 10:
                    addresses.add(line.strip())
        return addresses

    def _extract_social_links(self, soup: BeautifulSoup, base_url: str) -> Dict[str, List[str]]:
        links = {platform: [] for platform in self.SOCIAL_PLATFORMS}
        for a in soup.find_all('a', href=True):
            href = a['href'].lower()
            for platform, domains in self.SOCIAL_PLATFORMS.items():
                if any(d in href for d in domains):
                    full_url = urljoin(base_url, a['href'])
                    if full_url not in links[platform]:
                        links[platform].append(full_url)
        return links

    def _scrape_page(self, url: str) -> tuple:
        try:
            print(f"📄 Scraping: {url}")
            resp = self.session.get(url, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            for el in soup(['script','style','noscript']):
                el.decompose()
            text = soup.get_text()
            emails = self._extract_emails(text)
            phones = self._extract_phones(text)
            addresses = self._extract_addresses(text)
            social_links = self._extract_social_links(soup, url)
            return emails, phones, addresses, social_links
        except Exception as e:
            print(f"⚠️ Requests failed for {url}: {e}")
            return set(), set(), set(), {p: [] for p in self.SOCIAL_PLATFORMS}

    def scrape_main_website(self, website_url: str) -> Dict:
        urls_to_scrape = [website_url]  # main page
        main_emails, main_phones, main_addresses = set(), set(), set()
        main_social = {p: [] for p in self.SOCIAL_PLATFORMS}

        # Try priority pages first: contact/about
        try:
            resp = self.session.get(website_url, timeout=10)
            soup = BeautifulSoup(resp.text, 'html.parser')
            for a in soup.find_all('a', href=True):
                href = a['href']
                if any(k in href.lower() for k in self.PRIORITY_KEYWORDS):
                    full_url = urljoin(website_url, href)
                    if full_url not in urls_to_scrape:
                        urls_to_scrape.append(full_url)
        except:
            pass

        for url in urls_to_scrape:
            emails, phones, addresses, social_links = self._scrape_page(url)
            main_emails.update(emails)
            main_phones.update(phones)
            main_addresses.update(addresses)
            for p in main_social:
                main_social[p].extend([l for l in social_links[p] if l not in main_social[p]])

        return {
            "website": website_url,
            "emails": sorted(list(main_emails)),
            "phones": sorted(list(main_phones)),
            "addresses": sorted(list(main_addresses)),
            "social_links": {p: list(set(v)) for p,v in main_social.items()}
        }

    def scrape_from_gmaps(self, gmaps_url: str) -> Optional[Dict]:
        website = self.get_website_from_google_maps(gmaps_url)
        if website:
            return self.scrape_main_website(website)
        return None

    def save_to_json(self, data: Dict, filename='business_info4.json'):
        with open(filename,'w',encoding='utf-8') as f:
            json.dump(data,f,indent=2,ensure_ascii=False)
        print(f"💾 Data saved to {filename}")

def main():
    gmaps_url = input("📍 Enter Google Maps URL: ").strip()
    if not gmaps_url:
        print("❌ No URL provided")
        return
    crawler = FocusedCrawler(headless=True)
    result = crawler.scrape_from_gmaps(gmaps_url)
    if result:
        print(json.dumps(result, indent=2, ensure_ascii=False))
        crawler.save_to_json(result)
    else:
        print("❌ Could not scrape website")

if __name__ == "__main__":
    main()


🔍 Opening Google Maps: https://maps.app.goo.gl/6LH3qEbrVK2YKNSx7
✅ Found website: https://med.uth.edu/
📄 Scraping: https://med.uth.edu/
📄 Scraping: https://outlook.office.com/
📄 Scraping: https://med.uth.edu/about-us/
📄 Scraping: https://med.uth.edu/about-us/dual-degree-programs/
⚠️ Requests failed for https://med.uth.edu/about-us/dual-degree-programs/: HTTPSConnectionPool(host='med.uth.edu', port=443): Read timed out.
📄 Scraping: https://med.uth.edu/contact-us/
⚠️ Requests failed for https://med.uth.edu/contact-us/: HTTPSConnectionPool(host='med.uth.edu', port=443): Read timed out. (read timeout=10)
{
  "website": "https://med.uth.edu/",
  "emails": [],
  "phones": [
    "30033-4097",
    "713-500-51026431"
  ],
  "addresses": [],
  "social_links": {
    "facebook": [
      "https://www.facebook.com/mcgovernmed/"
    ],
    "instagram": [
      "https://www.instagram.com/mcgovernmed/"
    ],
    "linkedin": [
      "https://www.linkedin.com/school/mcgovern/"
    ],
    "twitter": [
  